# The Model
> Designing the model

- toc: true 
- badges: false
- comments: true
- hide: false
- categories: [jupyter]
- permalink: /the-model
- image: images/programming.png

The original `deshima-sensitivity` code follows the following flowchart

![old deshima code](images/Old_deshima.png)

The optical chain described in the flowchart consists of serial transmission calculations from 1 medium to another. This entire chain of stacked transformations of the spectral input power can be regarded as a single affine transformation of the input and is independent of the bandwith or amount of frequency bins. It will remain unchanged throughout the proposed modifications.

The power impinging on a detector for a specific filter bin is calculated by taking the spectral flux density at the center frequency $\nu_c$ and multiplying this with the bandwith $\Delta\nu$. This matches the model of the previous described box filter, assuming the flux density $\mathrm{psd}$ is flat over the entire bandwith. This is not the case however, as we shall see.

In [30]:
#hide
!pip install -q galspec numpy pandas plotly "deshima-sensitivity==0.3.0" scipy

import numpy as np
import pandas as pd
import plotly.express as px

import warnings
warnings.filterwarnings('ignore')

from thesis_plot import plot, filter_plot

# Page Specific
import galspec
from deshima_sensitivity.atmosphere import eta_atm_func
from scipy.stats import norm
from scipy.stats import cauchy, norm
from scipy.interpolate import interp1d

from plotly.subplots import make_subplots

In [31]:
#hide
def eta_filter_func(F,F_ch,FWHM):
    eta_filter = (cauchy.pdf(F,F_ch,FWHM*0.5)*FWHM*0.5*np.pi).T
    return eta_filter

In [32]:
#hide
h = 6.626e-34
k_b = 1.381e-23
e = 1.60217662 * 10**-19 # electron charge
c = 299792458.

## Filter distribution

1 filter channel has so far been approximated as a box integral, which isn't physically possible and therefore inaccurate. The actual filter shapes are described by a Lorentzian curve

$$\begin{equation}
\eta\left(\nu\right)=\frac{A\gamma^2}{\left(\nu-\nu_0\right)^2+\gamma^2}
\end{equation}$$

Where $\gamma$ is the half-width at half maximum (HWHM) value, for DESHIMA given as
$$\begin{equation}
\gamma=\frac{\nu_c}{2R}
\end{equation}$$
with $R=500$ the spectral resolution. As we have seen previously our box filters were defined by an efficiency constant labeled $\eta_0$. Together with the bandwidth, this describes the transmission of the box filter. If we define the full width at half maximum for the Lorentzian as the bandwidth of the Lorentzian, we would like to define the $A$ parameter such that the area of this $\mathrm{FWHM}$ is exactly the same as the area of the box filter.

In [33]:
#hide_input
def split_draw():
    N_on_3 = 200
    F_c=350
    HWHM=350/100
    F=np.linspace(F_c-3*HWHM,F_c+3*HWHM,3*N_on_3)
    Lorentzian_left=cauchy.pdf(F[0:N_on_3],F_c,HWHM)*HWHM*np.pi
    Lorentzian_center=cauchy.pdf(F[N_on_3:2*N_on_3],F_c,HWHM)*HWHM*np.pi
    Lorentzian_right=cauchy.pdf(F[2*N_on_3:3*N_on_3],F_c,HWHM)*HWHM*np.pi
    zeros=np.zeros(N_on_3)
    Lorentzian_outband=np.concatenate((Lorentzian_left,zeros,Lorentzian_right))
    Lorentzian_inband=np.concatenate((zeros, Lorentzian_center, zeros))
    
    box_center=np.ones(N_on_3)*np.pi/4
    box_inband= np.concatenate((zeros, box_center, zeros))
    
    df = pd.DataFrame({'F': F,'In band': Lorentzian_inband, 'Out band': Lorentzian_outband, 'Box approximation': box_inband})
    df = df.set_index('F')
    plot(df,axis_labels=('$\\nu\:\mathrm{[GHz]}$','$\eta$',''),hover_labels=('nu [GHz]','transmission',''),y_range=(0.01,1.05),e_notation=(False, False), fill='tozeroy')
split_draw()

$$\begin{equation}
\int_{\nu_0-\gamma}^{\nu_0+\gamma}\frac{A\gamma^2}{\left(\nu-\nu_0\right)^2+\gamma^2}d\nu=\frac{\pi}{2}\gamma A\Leftrightarrow A = \frac{4}{\pi}\eta_0
\end{equation}$$

Meaning our filters are given by 

$$\begin{equation}
\eta\left(\nu\right)=\frac{4}{\pi}\frac{\eta_0\gamma^2}{\left(\nu-\nu_0\right)^2+\gamma^2}
\end{equation}$$

For atmospheric loading, which as you might recall is the main source of the noise, the channel is of course loaded over the entire filter spectrum: both in and out band. Luckily this is as simple as widening the effective bandwidth with a constant:

$$\begin{equation}
\int_{-\infty}^{\infty}\frac{A\gamma^2}{\left(\nu-\nu_0\right)^2+\gamma^2}d\nu=\pi\gamma A=2\int_{\nu_0-\gamma}^{\nu_0+\gamma}\frac{A\gamma^2}{\left(\nu-\nu_0\right)^2+\gamma^2}d\nu
\end{equation}$$

Meaning that the full band effective bandwidth is exactly twice the in band effective bandwidth.

### Noise Equivalent Power
Redoing the calculation of the noise equivalent power for a flat $\mathrm{PSD}$, but with proper Lorentzian filters this time, results in:

$$\begin{equation}
\begin{split}
\mathrm{NEP}_{\tau=0.5\mathrm{s},\mathrm{ph}}^2&=2\int_0^\infty h\nu\eta(\nu)\mathrm{PSD}+\eta^2(\nu)\mathrm{PSD}^2d\nu\\
&=2\mathrm{PSD}\int_0^\infty h\nu\eta(\nu)d\nu+2\mathrm{PSD}^2\int_0^\infty \eta^2(\nu)d\nu \\
&\approx8\mathrm{PSD}h\nu_0\eta_0\gamma + \frac{16}{\pi}\mathrm{PSD}^2\eta_0^2\gamma \\
&=4\mathrm{PSD}h\nu_0\eta_0\mathrm{FWHM}+\frac{8}{\pi}\mathrm{PSD}^2\eta_0^2\mathrm{FWHM}
\end{split}
\end{equation}$$

where an approximation is used for the first term, assuming $\Delta\nu\ll\nu$. Because the calculations are done for a constant $\mathrm{PSD}$, the filter gets loaded across the full band, meaning the effective bandwidth $\Delta\nu=2\mathrm{FWHM}$. Putting this in the equation for the $\mathrm{NEP}_{\tau=0.5\mathrm{s},\mathrm{ph}}$ yields:

$$\begin{equation}
\begin{split}
\mathrm{NEP}_{\tau=0.5\mathrm{s},\mathrm{ph}} &= 2\mathrm{PSD}\eta_0\Delta\nu+\frac{4}{\pi}\mathrm{PSD}^2\eta_0^2\Delta\nu \\
&=2P_\mathrm{KID}h\nu_0 + \frac{4}{\pi}\frac{P^2_\mathrm{KID}}{\Delta\nu}
\end{split}
\end{equation}$$
 
Which means the second term, the bunching term, is a factor of $\pi/2$ smaller than is the case for narrow-bandwidth approximation described in {% cite zmuidzinas_2003 %} and used in {% cite endo_2019%}.

## Non-flat Power Spectrum Densities

We know the $\mathrm{NEP}$ is proportional to the $\mathrm{PSD}$ impinging on the detector, and we have also seen that the atmospheric loading is a much bigger contribution to the spectral flux density than any atmospheric source. let's take a look at how this atmospheric loading varies over the bin width. In the figures below the actual target frequency channels from DESHIMA 2.0 are taken and compared to the atmospheric model measured at ASTE, taken from `deshima-sensitivity`.

In [34]:
#hide_input
F_step = 5000
fMin = 200
fMax = 500

pwv = 1

eta_A = 0.6 # aperture efficiency
telescope_diameter = 10
A_p = (telescope_diameter/2)**2 * np.pi # telescope physical area
A_e = eta_A * A_p  # telescope effective area

T_p_atm = 273.

F = np.linspace(fMin,fMax,F_step)
nu=F
T=np.array([2.725])

# Calculate atmospheric loading and put in DF
eta_atm = eta_atm_func(F=F, pwv=pwv, EL=90)
S_sky = (1-eta_atm) * T_p_atm * 1e26 * 2 *k_b /A_e

df_P=pd.DataFrame({'nu':F,'P':S_sky})
df_P.set_index('nu',inplace=True)

# Calculate spectrometer center frequencies and put in DF
i=np.arange(0,347)
R=500

F=np.empty((347,F_step))
F_name=[None]*347

F_center=220*np.power(1+1/R,i)
for idx, F_bin in np.ndenumerate(F_center):
    F[idx]= ((nu>F_bin*(1-1/(2*R)))*(nu<F_bin*(1+1/(2*R))))*np.pi/4
    F_name[idx[0]]="{} ({:.2f} GHz)".format(str(idx[0]),F_bin)

df_box_bins = pd.DataFrame(F.T)
df_box_bins.columns = F_name#np.round(F_center,decimals=2)
df_box_bins['nu']=nu
df_box_bins = df_box_bins.set_index('nu')

# Make plot managable
start_F = 285
stop_F=290
start_nu= np.argmax(nu>start_F)
stop_nu = np.argmax(nu>stop_F)
start_bin=np.argmax(F_center>start_F)
stop_bin=np.argmax(F_center>stop_F)+1

filter_plot(df_P.iloc[start_nu:stop_nu,0],df_box_bins.iloc[start_nu:stop_nu,start_bin:stop_bin],mode='box')


For the channels where the atmospheric loading is reasonably constant over the bandwidth this approximation is fine, however when the spectral flux density peaks to twice it's local background level the box filters are obviously a poor approximation for reality. The atmospheric loading is averaged over the bandwidth before being multiplied with it in the current version of `deshima-sensitivity`, in essence integrating it over the box filter. However, this crude approximation clearly fails when looking at the actual Lorentzian filters.

In [35]:
#hide_input

# Model profile
F_vect = cauchy.pdf(nu[np.newaxis].T,F_center,F_center/R/2)*F_center/R/2*np.pi
F = F_vect.T

df_bins = pd.DataFrame(F.T)
df_bins.columns = F_name
df_bins['nu']=nu
df_bins = df_bins.set_index('nu')

# Plot

filter_plot(df_P.iloc[start_nu:stop_nu,0],df_bins.iloc[start_nu:stop_nu,start_bin:stop_bin])

The peak at $286.1\:\mathrm{GHz}$ is obviously loading channel 131 and 132, but due to the wide profile of a Lorentzian even 130 and 133 are receiving energy from the peak in flux density.

The total power impinged on the detector can be approximated by calculating the spectral power arriving at the detector not just at the center frequencies of the bins $\nu_0$, but by expanding the number of frequencies calculated to some amount of *integration bins* and calculating the spectral power at these frequencies. For an integration frequency $\nu_i$ and a channel $j$, the effective power loading by that frequency on that channel is given by:

$$\begin{equation}
P_{\mathrm{KID},i,j} = \eta\left(\nu_i,j\right)\mathrm{psd}\left(\nu_i\right)\Delta\nu_i\label{p_kid_i}
\end{equation}$$

with $\Delta\nu_i$ the bandwidth for that frequency. The total power impinged on a filter channel $P_{\mathrm{KID},j}$ is then given by:

$$\begin{equation}
P_{\mathrm{KID},j}=\sum_i P_{\mathrm{KID},i,j} = \sum_i \eta\left(\nu_i,j\right)\mathrm{psd}\left(\nu_i\right)\Delta\nu_i \label{p_kid}
\end{equation}$$

To make this more visual, take a look at the figure below

In [36]:
#hide_input

# Closer zoom
start_F_2 = 285
stop_F_2=287
start_nu_2= np.argmax(nu>start_F_2)
stop_nu_2 = np.argmax(nu>stop_F_2)
start_bin_2=np.argmax(F_center>start_F_2)
stop_bin_2=np.argmax(F_center>stop_F_2)+1

filter_plot(df_P.iloc[start_nu_2:stop_nu_2,0],df_bins.iloc[start_nu_2:stop_nu_2,start_bin_2:stop_bin_2],'stem',[start_F_2,stop_F_2])

The calculated flux spectrum arriving at the filter will be calculated as is done in the current version of the `deshima-sensitivity` package, but using a finer frequency resolution. This spectrum will then arrive at the filter, where for each channel a weighted sum will be taken over all integration bins, resulting in a total power impinged on the detector.

## Noise Equivalent Power

As seen, the photon noise equivalent power for a single DESHIMA channel $\mathrm{NEP}_{\tau=0.5\mathrm{s},\mathrm{ph},j}$is given by{%cite zmuidzinas_2003%}:

$$\begin{equation}
\mathrm{NEP}_{\tau=0.5\mathrm{s},\mathrm{ph},j}^2=2\int_0^\infty h\nu\eta(\nu)\mathrm{PSD}+\eta^2(\nu)\mathrm{PSD}^2d\nu
\end{equation}$$

Again taking the filter efficiency $\eta\left(\nu_i,j\right)$ and $\mathrm{PSD}$ constant over a single integration frequency, and multiplying $\mathrm{PSD}$ with the bandwidth to get the power impinged by an integration bin on the channel $P_{\mathrm{KID},i}$ we can easily take the Riemann sum of this integral as

$$\begin{equation}
\mathrm{NEP}_{\tau=0.5\mathrm{s},\mathrm{ph},j}^2=2\sum_i\left( h\nu_iP_{\mathrm{KID},i}+\frac{P_{\mathrm{KID},i}^2}{\Delta\nu_i}\right)\label{nep_ph}
\end{equation}$$

The recombination noise $\mathrm{NEP_{R}}$ is given by

$$\begin{equation}
\mathrm{NEP}_{\mathrm{R},j}^2=4\Delta_\mathrm{Al}\frac{P_{\mathrm{KID},j}}{\eta_\mathrm{pb}}=4\Delta_\mathrm{Al}\frac{\sum_i{P_{\mathrm{KID},j,i}}}{\eta_\mathrm{pb}}
\end{equation}$$

Which results in a total $\mathrm{NEP}_{\tau=0.5\mathrm{s}}$ of

$$\begin{equation}
\mathrm{NEP}_{\tau=0.5\mathrm{s}} = \sqrt{2\sum_i\left( h\nu_iP_{\mathrm{KID},i}+\frac{P_{\mathrm{KID},i}^2}{\Delta\nu_i}\right) + 4\Delta_\mathrm{Al}\frac{\sum_i{P_{\mathrm{KID},j,i}}}{\eta_\mathrm{pb}}}\label{total_NEP}
\end{equation}$$

### Photon Bunching

But what about photon bunching? I have shown that the coherence time and the bandwidth are inversely proportional. By decreasing the size of the integration bins we have also decreased the bandwidth and therefore increased the coherence time. Thankfully the integration time of our definition of the noise equivalent power, $0.5\:\mathrm{s}$, is orders of magnitude bigger than the coherence time {% cite morgan_1966%}. As we have seen earlier, if the photons are sampled at orders of magnitude higher than the coherence time, doubling the coherence time (halving the bandwith) has no effect. You might say we are hiding the non-stochastic effects of photon bunching in the coarseness of our Riemann sum.

Now the $\mathrm{NEP}$ is simply defined at $\tau=0.5\:\mathrm{s}$, but in measurements the integration time can vary. The $\mathrm{NEP}$ is scaled accordingly:

$$\begin{equation}
\mathrm{NEP}_{\tau} = \frac{1}{\sqrt{\tau}}\sqrt{\sum_i\left( h\nu_iP_{\mathrm{KID},i}+\frac{P_{\mathrm{KID},i}^2}{\Delta\nu_i}\right) + 2\Delta_\mathrm{Al}\frac{\sum_i{P_{\mathrm{KID},j,i}}}{\eta_\mathrm{pb}}}=\frac{1}{\sqrt{2\tau}}\mathrm{NEP}_{\tau=0.5\mathrm{s}}
\end{equation}$$
But this scaling doesn't hold indefinitely. I conjecture that at some point the incoming photons are sampled at such a short sampling interval that they become correlated, as we have seen previously. This comes with a drop in noise as in the limit either zero or one photon can arrive. This is shown qualitatively in the figure below.

In [37]:
#hide_input
min_exp=-9
max_exp=1
N_points=500
limit_scale=1
tau=np.logspace(min_exp,max_exp,N_points)
NEP_approx=1/np.sqrt(2*tau)

rampdown=(np.exp(-10**min_exp/tau))
rampup=1/rampdown

NEP_real=rampdown*NEP_approx
df_NEP=pd.DataFrame({'tau':tau,'Approximation':NEP_approx,'Limit': NEP_real})
df_NEP.set_index('tau',inplace=True)
plot(df_NEP,axis_labels=('$\\tau$','$\\frac{\mathrm{NEP}_\\tau}{\mathrm{NEP}_{\\tau=0.5\mathrm{s}}}$',''),hover_labels=('Integration time','Normalized NEP',''),plot_type='loglog',e_notation=(False, False))

## The Filter Matrix

From eq \eqref{p_kid_i} we can see that we need a two dimensional function $\eta\left(\nu_i,j\right)$ to go from a one dimensional $\mathrm{PSD}$ to a power spectrum for each filter channel. We do this programmatically by calculating a matrix that takes in a vector containing the $\mathrm{PSD}$ at each integration frequency and uses that to calculate the power loading by this $\mathrm{PSD}$ on each channel.

Shown in the figure below is a visualization of how this filter matrix might look like.

In [38]:
#hide_input
N_bins = 20
spacing = 50
np.random.seed(0)
random_cauchy = cauchy.rvs(size=1000)[np.newaxis].T
ones=np.ones(N_bins)
n=np.arange(N_bins,0,-1)+2

lorentzian_random_df = pd.DataFrame((random_cauchy * ones) + n*spacing)

fig = px.violin(lorentzian_random_df,
                orientation='h',
                labels={'value':'Integration Frequencies','variable': 'Channels'},
                template = 'plotly_white',
                points=False,)
fig.update_traces(side='positive',width=4,hoveron="points")
fig.update_xaxes(showticklabels=False, showgrid=False,range=[0,(N_bins+4)*spacing])
fig.update_yaxes(showticklabels=False, showgrid=False,zeroline=False)

fig.show()

With this framework in place it is easy to swap out the generated filter with another, more realistic model of the filters. The figure below shows a simulated filter transmission curve for the DESHIMA 2.0 chip.

![DESHIMA 2.0 filter-profile](images/rainbow_raw_log_Spectral.png)*A simulated filter profile of the DESHIMA 2.0 spectrometer*

As you can see, the filters aren't all equally efficient and wide. In the figure below all 346 bins and the corresponding values of $R$ are plotted, with the size corresponding to the transmission.

In [39]:
#hide_input
fit_data=pd.read_csv("fit_data_filter_transmission.csv")
fit_data["F_center"] = fit_data["F_center"]
fit_data["HWHM"] = fit_data["HWHM"]
fit_data['idx']=np.arange(len(fit_data))+1
fig = px.scatter(fit_data,
                 x='F_center',
                 y="R",
                 custom_data=['idx','chi_square'],
                 color='F_center',
                 size="peak_value",
                 size_max=30,
                 color_continuous_scale=px.colors.sequential.Rainbow_r,
                 labels={"F_center":"$\\nu_c\:\mathrm{[GHz]}$","R": "$R$"},
                 template='plotly_white')

fig.update_layout(coloraxis_showscale=False,
                 hoverlabel={'bgcolor': "white", 'font_size': 14})
fig.update_traces(hovertemplate="<b>Bin: %{customdata[0]} (%{x:.2f} GHz)</b><br>R: %{y:.0f}<br>transmission: %{marker.size:.3f}<br>Chi-squared:%{customdata[1]:.2e}<extra></extra>")
fig.show()

While using a perfect Lorentzian approximation for the filter profiles for each channel is better than the center frequency sampling that is done now, it might be more advantageous to be able to use simulated or measured profiles. The filter matrix will therefore, depending on the users choice, be either generated via Lorentzian curves or loaded in via a file.

## Transforming the calculated noise

Once the $\mathrm{NEP}$ has been calculated, it needs to be transformed back into something more usable to define the sensitivity of the instrument. We do this by calculating the *source coupling* through a quantity named the Noise Equivalent Source Flux $\mathrm{NEF}$. The $\mathrm{NEF}$ is defined as the amount of power a source needs to emit per second to equal the $\mathrm{NEP}$ in strength and is therefore our sensitivity: a source with a flux lower than the $\mathrm{NEF}$ will be hidden in the noise of the measurement data. Take note that the $\mathrm{NEF}$ is defined for an integration time of $\tau=1\:\mathrm{s}$, rather than $\tau=0.5\:\mathrm{s}$ as is the case for the $\mathrm{NEP}$. The $\mathrm{NEF}$ is given by:
$$\begin{equation}
\mathrm{NEF} = \frac{\mathrm{NESP}_{\tau=0.5\mathrm{s}}}{\sqrt{2}A_g}=\frac{\mathrm{NEP_{inst,{\tau=0.5\mathrm{s}}}}}{\eta_\mathrm{sw}\sqrt{2}A_g}=\frac{\mathrm{NEP}_{\tau=0.5\mathrm{s}}}{\eta_\mathrm{inst}\eta_\mathrm{sw}\sqrt{2}A_g}
\end{equation}$$

with $A_g$ the area of the telescope, $\eta_\mathrm{sw}$ the aggregate efficiency from the source to the window of the cryostat chamber DESHIMA is housed in and $\eta_\mathrm{inst}$ the instrument efficiency. The first two remain unchanged throughout the modifications discussed in this chapter, however $\eta_\mathrm{inst}$ is dependent on $\eta_\mathrm{filter}$ and is therefore modified. 

Since the calculation of the $\mathrm{NEP}_{\tau=0.5\mathrm{s}}$ as in \eqref{total_NEP} collapses the noise equivalent power down to a single value per channel, we can't use the efficiency as calculated for the integration bins to go back through the system in order to calculate the source coupling. Instead there are two different approximations we can take to calculate the source coupling: an in-band and a full band approximation


### In-band Source Coupling

The in-band approximation is used for spectral sources. A spectral source is defined as an object that transmits only within the bandwidth of a single filter channel, where we define the bandwidth as the $\mathrm{FWHM}$

In [40]:
#hide_input
# Model Spectral source
bin_n = 233
F_source=220*np.power(1+1/R,bin_n)
spectral_source=norm.pdf(nu,F_source,F_source/3000)*0.033

# Zoom in
# Make plot managable
start_F = 347.5
stop_F=352.5
start_nu_3= np.argmax(nu>start_F)
stop_nu_3 = np.argmax(nu>stop_F)
start_bin_3=np.argmax(F_center>start_F)
stop_bin_3=np.argmax(F_center>stop_F)+1

df_spectral_source=pd.DataFrame({'nu':nu,'flux':spectral_source}).set_index('nu')
filter_plot(df_spectral_source.iloc[start_nu_3:stop_nu_3,0],df_bins.iloc[start_nu_3:stop_nu_3,start_bin_3:stop_bin_3],y_axis_range=[0,0.15],spectrum_title='Spectral Source')

In order to get a single value of an efficiency, let's say $\eta_\mathrm{inst}$, over the full in-band bandwidth, I take a weighted average over the $\mathrm{FWHM}$ where the weights are given by the filter.

$$\begin{equation}
\eta_\mathrm{inst,j}=\frac{\int_{\nu_j-\gamma}^{\nu_j+\gamma}\eta_\mathrm{inst}\left(\nu\right)\eta_\mathrm{filter}\left(\nu,j\right)d\nu}{\int_{\nu_j-\gamma}^{\nu_j+\gamma}\eta_\mathrm{filter}\left(\nu,j\right)d\nu}
\end{equation}$$

This ensures that the efficiency per channel is normalized such that I can use the box-filter approximation on our way back through the system.

In [41]:
#hide_input
filter_plot(df_spectral_source.iloc[start_nu_3:stop_nu_3,0],df_box_bins.iloc[start_nu_3:stop_nu_3,start_bin_3:stop_bin_3],mode='box',y_axis_range=[0,0.15],spectrum_title='Spectral Source')

### Full-band Source Coupling

Besides spectral sources, astronomical objects also radiate a continuous spectrum, through blackbody radiation:

In [42]:
#hide_input
F_gal, S_gal = galspec.spectrum(luminosity=13.7, redshift=6.43, fLow=fMin, fHigh=fMax, numFreqBins=F_step, linewidth=600)

df_continuum_source=pd.DataFrame({'nu':nu,'flux':S_gal}).set_index('nu')
filter_plot(df_continuum_source.iloc[start_nu_3:stop_nu_3,0],df_bins.iloc[start_nu_3:stop_nu_3,start_bin_3:stop_bin_3],y_axis_range=[0,0.07],spectrum_title='Spectral Source')

Because this continuum source is as near as makes no difference constant over the frequency bin, we can model it's transmission by another box-filter, with double the bandwidth of the spectral case, as I have derived previously. 

In this case, the efficiencies are calculated using another weighted average, weighted over the full band this time 

$$\begin{equation}
\eta_\mathrm{inst,j}=\frac{\int_0^\infty\eta_\mathrm{inst}\left(\nu\right)\eta_\mathrm{filter}\left(\nu,j\right)d\nu}{\int_0^\infty\eta_\mathrm{filter}\left(\nu,j\right)d\nu}
\end{equation}$$

## The Final Model

Using these two approximations, the NEP can be transformed back through the system and to the sky again. This means that the total model looks like this:
![new deshima code](images/New_deshima.png)

<div style="display: flex;justify-content: space-between">
    <a href="{{site.baseurl}}/photon-statistics" >
        <div style="display: inline;font-size: x-large;padding: .5em;width:auto;"
             class="Box box-shadow-medium rounded-1 col-12">Previous Chapter</div>
    </a>
    <a href="{{site.baseurl}}/results">
        <div style="display: inline;font-size: x-large;padding: .5em;width:auto;"
             class="Box box-shadow-medium rounded-1 col-12">Next Chapter</div>
    </a>
</div>

## Bibliography
{% bibliography --cited %}